### Import Library

In [1]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import c3d
import importlib
from typing import List

# Import helpers
sys.path.append('/app/hpe-core/datasets/vicon_processing/v2')
import helpers

# Import bimvee
sys.path.append('./submodules/bimvee')
from bimvee.importIitYarp import importIitYarp

### Load parameters

In [2]:
vicon_c3d_file: str = '/data/test7/VICON/test701.c3d'
events_folder: str = '/data/test7/left'
calib_path: str = '/data/test7/calib_left.txt'
period: float = 1.0 / 100  # vicon frequency: 100HZ
input_label_tag_file: str = '../scripts/config/labels_tags_vicon.yml'
output_label_yml_file: str = 'labels_all_test7_tmp_left.yml'

### Import event data

In [3]:
v_data: dict = importIitYarp(filePathOrName=events_folder, template=None, wrapTime=0.0)
e_ts: np.ndarray = v_data['data']['left']['dvs']['ts']
e_us: np.ndarray = v_data['data']['left']['dvs']['x']
e_vs: np.ndarray = v_data['data']['left']['dvs']['y']

importIitYarp trying path: /data/test7/left


100%|██████████| 123435/123435 [05:14<00:00, 392.57it/s]


Examining info.log: /data/test7/left/info.log


### Import C3D data

In [4]:
c3d_data = c3d.Reader(open(vicon_c3d_file, 'rb'))
points_3d: dict = {}
for i, points, analog in c3d_data.read_frames():
    points_3d[i] = points

marker_t = np.linspace(0.0, c3d_data.frame_count / c3d_data.point_rate, 
                            c3d_data.frame_count, endpoint=False)

/app/hpe-core/datasets/vicon_processing/v2/.venv/lib/python3.10/site-packages/c3d/c3d.py:1219: UserWarning: No analog data found in file.
  warnings.warn('No analog data found in file.')


In [5]:
c3d_data.point_labels

array(['stereoatis:cam_right          ', 'stereoatis:cam_back           ',
       'stereoatis:cam_left           ', 'monitor:Origin_mon_top_right  ',
       'monitor:mon_bottom_right      ', 'monitor:mon_bottom_left       ',
       'monitor:mon_top_left          ', 'monitor:base_origin_1         ',
       'monitor:base_2                ', 'monitor:base_3                ',
       'monitor:base_4                ', 'chair:origin_right_top        ',
       'chair:right_mid               ', 'chair:right_bottom            ',
       'chair:left_top                ', 'chair:left_bottom             ',
       'box2:top_right_Origin         ', 'box2:bottom_right             ',
       'box2:bottom_left              ', 'box2:top_left                 ',
       'box1:top_1_origin             ', 'box1:top_2                    ',
       'box1:top_3                    ', 'box1:top_4                    ',
       'box1:bottom_1                 ', 'box1:bottom_2                 ',
       'box1:bottom_3    

### Import calibration data

In [6]:
calib = np.genfromtxt(calib_path, delimiter=" ", skip_header=1, dtype=object)

calib_dict = {}
keys = calib[:, 0].astype(str)
vals = calib[:, 1].astype(float)
calib_dict = {
    key: value for key, value in zip(keys, vals)
}
cam_res = np.int64([calib_dict['h'], calib_dict['w']])
cam_res_rgb = np.append(cam_res, 3)
# intrinsic
K = np.array([
    [calib_dict['fx'], 0.0, calib_dict['cx']],
    [0.0, calib_dict['fy'], calib_dict['cy']],
    [0.0, 0.0, 1.0]
])
# camera distortion
D = np.array([calib_dict['k1'], calib_dict['k2'], calib_dict['p1'], calib_dict['p2']])


### Visualize event data as time window

In [22]:
img = np.ones(cam_res, dtype = np.uint8)*255

ft = e_ts[0]
for i in range(0,len(e_ts)):
    #clear the frame at intervals
    if e_ts[i] >= ft:
        cv2.imshow('Visualisation', img)
        k = cv2.waitKey(np.int64(period*1000))
        if k == 27:  # ESC key to stop
            break
        img = np.ones(cam_res, dtype = np.uint8)*255
        text = f"t = {ft:.6f}s"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        thickness = 2
        color = (0, 0, 0)
        (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)
        x = img.shape[1] - text_width - 10
        y = text_height + 10
        cv2.putText(img, text, (x, y), font, font_scale, color, thickness, cv2.LINE_AA)
        ft = ft + period
    img[e_vs[i],e_us[i]] = 0
    
cv2.destroyAllWindows()

### Visualize marker positions

In [9]:
%matplotlib tk
marker_points = helpers.marker_p(c3d_data.point_labels, points_3d.values(), 'stereoatis:cam_right')
plt.plot(marker_t, marker_points[:, 0:3])
plt.show()

### Manual adjustment for delay init

In [7]:
delay = 0.365     # adjust accordingly

In [ ]:
#TODO: add cell asking user to input the translation?

In [ ]:
#TODO: add cell to manually adjust rotation -> roll, pitch, yaw angles

In [ ]:
#TODO: move here cell to correct delay by going through all frames

### Label sequence

In [ ]:
importlib.reload(helpers)
from helpers import DvsLabeler

labels_path = os.path.join(os.path.dirname(vicon_c3d_file), output_label_yml_file)

time_tags, event_indices = helpers.calc_indices(e_ts, period)

img = np.ones(cam_res, dtype = np.uint8)*255

labeler = DvsLabeler(img_shape=(720, 1280, 3))
labeler.label_data(e_ts, e_us, e_vs, event_indices, time_tags, period, input_label_tag_file)
labeler.save_labeled_points(labels_path)

print("Saved labeled points at:", labels_path)

#instead of saving the points save the obtained transformation matrix and the delay (block called "load labels and calculate transformation matrix")

cv2.destroyAllWindows()

KeyboardInterrupt: 

### Load labels and calculate transformation matrix from system to camera using scipy optimizer

In [8]:
from scipy.optimize import least_squares

def reprojection_error(params, Ps, pc, K, dist):
    """
    params: 6 dimensions (rotation_vector [3], translation [3]) Optimization target
    Ps: (N, 3) 3D points (system coordinates)
    pc: (N, 2) 2D points (image coordinates)
    K: (3, 3) camera matrix
    dist: (5,) distortion parameters (OpenCV format)
    """
    rvec = params[:3]
    tvec = params[3:6]

    # Project 3D system points to 2D image points
    projected_points, _ = cv2.projectPoints(Ps, rvec, tvec, K, dist)
    projected_points = projected_points.reshape(-1, 2)

    return (projected_points - pc).ravel()  # Flatten and return


def estimate_Tstoc(Ps, pc, K, dist, init_params=None):
    """
    Ps: (N, 3) 3D points (system coordinates)
    pc: (N, 2) 2D points (image coordinates)
    K: (3, 3) camera matrix
    dist: (5,) distortion parameters
    return: (4, 4) system→camera coordinate transformation matrix Tstoc
    """

    # Initial values: zero rotation, zero translation
    if init_params is None:
        # If no initial parameters are provided, set them to zero
        init_params = np.zeros(6)

    # Minimize (choose LM method, etc.)
    res = least_squares(
        reprojection_error,
        init_params,
        args=(Ps, pc, K, dist),
        method='lm'  # Levenberg-Marquardt
    )

    rvec_opt = res.x[:3]
    tvec_opt = res.x[3:6]
    R_opt, _ = cv2.Rodrigues(rvec_opt)

    # Construct homogeneous transformation matrix
    T = np.eye(4)
    T[:3, :3] = R_opt
    T[:3, 3] = tvec_opt

    return T

In [9]:
importlib.reload(helpers)
from helpers import ViconHelper

# Define vicon helper
print("Delay: ", delay) # Please adjust delay manually
vicon_helper = ViconHelper(marker_t, points_3d, delay, c3d_data.frame_count, c3d_data.point_rate, c3d_data.point_labels, True, True) # if needed create the function to call instead of None

# Load labeled points
labels_path = os.path.join(os.path.dirname(vicon_c3d_file), output_label_yml_file)
labeled_points = helpers.read_points_labels(labels_path) # read labeled points from the yaml file

# extract the frames_id from the labeled points
frames_id = vicon_helper.get_frame_time(labeled_points['times'])
vicon_points = vicon_helper.get_vicon_points_interpolated(labeled_points)
print("First few label dicts from YAML:")
for i, d in enumerate(labeled_points['points']):
    print(f"Frame {i}: {d}")

# Create lists of transformation matrix from world to system
system_points = []
image_points = []
Ts_system_to_camera: List[np.ndarray] = []
Ts_world_to_system = vicon_helper.compute_camera_marker_transforms()

# Collect correspondences from 3D system points to 2D image points
for idx, (dvs_frame, vicon_frame) in enumerate(zip(labeled_points['points'], vicon_points['points'])):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            w_p = vicon_frame[l]
            w_ph = np.append(w_p, 1.0)  # homogeneous coordinates
            i_p = [
                dvs_frame[l]['x'],
                dvs_frame[l]['y'] #,
                # 1.0
            ]
            frame_id = vicon_points['frame_ids'][idx]
            p_sys = Ts_world_to_system[frame_id] @ w_ph  # convert to system coordinates
            system_points.append(p_sys[:3])
            image_points.append(i_p)
        except Exception as e:
            print("The stored image labels probably don't match with the vicon labels used.")
            print(e)
system_points = np.array(system_points, dtype=np.float64)
image_points = np.array(image_points, dtype=np.float64)
image_points = image_points[:, :2]



Delay:  0.365
out {'points': [{'stereoatis:cam_right': array([-436.5234 ,  790.85315,  879.18604], dtype=float32), 'stereoatis:cam_back': array([-494.32202,  855.09015,  919.47534], dtype=float32), 'stereoatis:cam_left': array([-479.50995,  807.1114 ,  857.0858 ], dtype=float32)}, {'stereoatis:cam_right': array([-436.71155,  790.6289 ,  878.95746], dtype=float32), 'stereoatis:cam_back': array([-494.32016,  855.094  ,  919.4863 ], dtype=float32), 'stereoatis:cam_left': array([-479.55362,  807.0915 ,  857.0675 ], dtype=float32)}, {'stereoatis:cam_right': array([-436.28622,  791.00055,  879.47644], dtype=float32), 'stereoatis:cam_back': array([-494.31198,  855.1035 ,  919.4934 ], dtype=float32), 'stereoatis:cam_left': array([-479.54333,  807.1374 ,  857.12134], dtype=float32)}, {'stereoatis:cam_right': array([-436.49933,  790.81195,  879.1287 ], dtype=float32), 'stereoatis:cam_back': array([-494.31723,  855.12244,  919.48663], dtype=float32), 'stereoatis:cam_left': array([-479.5291 ,  807

In [10]:
#TODO: change this so that it is not hardcoded bit instead generated by user inputs

# Create initial guess for the optimization
from scipy.spatial.transform import Rotation
init_T = np.array([[ 5.54118388e-01, -6.46688971e-01, 5.24162298e-01, -1.25350531e+02],
                   [-3.65516271e-01, -7.5474063>6e-01, -5.44760888e-01, -8.06780263e+00],
                   [ 7.47897430e-01, 1.10272191e-01, -6.54591006e-01, -9.41323412e+01],
                   [ 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]) # Specify this value
r_vec = Rotation.from_matrix(init_T[:3, :3]).as_rotvec()
t_vec = init_T[:3, 3]
r_vec, t_vec, init_T
init_param = np.concatenate((r_vec, t_vec))
init_param

array([   2.14143154,   -1.1316251 ,    0.91920865, -125.350531  ,
         -8.06780263,  -94.1323412 ])

In [11]:
T_w2s_opt = estimate_Tstoc(system_points, image_points, K, D, init_param)
T_w2s_opt

array([[  0.58430039,  -0.64104819,   0.49764473,  47.64002178],
       [ -0.40405576,  -0.76160312,  -0.50665534,  -2.76594525],
       [  0.70379827,   0.09496269,  -0.70402421, -80.15438457],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

### Project points in dynamic scene

In [25]:
importlib.reload(helpers)

marker_names = ['stereoatis:cam_right          ', 'stereoatis:cam_back           ',
       'stereoatis:cam_left           ', 'monitor:Origin_mon_top_right  ',
       'monitor:mon_bottom_right      ', 'monitor:mon_bottom_left       ',
       'monitor:mon_top_left          ', 'monitor:base_origin_1         ',
       'monitor:base_2                ', 'monitor:base_3                ',
       'monitor:base_4                ', 'chair:origin_right_top        ',
       'chair:right_mid               ', 'chair:right_bottom            ',
       'chair:left_top                ', 'chair:left_bottom             ',
       'box2:top_right_Origin         ', 'box2:bottom_right             ',
       'box2:bottom_left              ', 'box2:top_left                 ',
       'box1:top_1_origin             ', 'box1:top_2                    ',
       'box1:top_3                    ', 'box1:top_4                    ',
       'box1:bottom_1                 ', 'box1:bottom_2                 ',
       'box1:bottom_3                 ', 'box1:bottom_4                 ']
projected_points = helpers.project_vicon_to_event_plane_dynamic(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, video_record = False, D=D
)

Starting correction with i_events: 0.000s, tic_events: 0.370s
Starting correction with i_markers: 0.000s, tic_markers: 0.005s


### Correct Delay by going though all frames

In [17]:
importlib.reload(helpers)

#TODO: modify function

marker_names = ['stereoatis:cam_right          ', 'stereoatis:cam_back           ',
       'stereoatis:cam_left           ', 'monitor:Origin_mon_top_right  ',
       'monitor:mon_bottom_right      ', 'monitor:mon_bottom_left       ',
       'monitor:mon_top_left          ', 'monitor:base_origin_1         ',
       'monitor:base_2                ', 'monitor:base_3                ',
       'monitor:base_4                ', 'chair:origin_right_top        ',
       'chair:right_mid               ', 'chair:right_bottom            ',
       'chair:left_top                ', 'chair:left_bottom             ',
       'box2:top_right_Origin         ', 'box2:bottom_right             ',
       'box2:bottom_left              ', 'box2:top_left                 ',
       'box1:top_1_origin             ', 'box1:top_2                    ',
       'box1:top_3                    ', 'box1:top_4                    ',
       'box1:bottom_1                 ', 'box1:bottom_2                 ',
       'box1:bottom_3                 ', 'box1:bottom_4                 ']
projected_points = helpers.fix_delay(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D
)

# delay fast -> 0.388 for monitor, shift for everything else

Space pressed, visualization paused at markers: 3.980s, events: 4.345s
Delay decreased to: 0.355s (step: 0.010s)
Updated frame with new delay: markers at 3.980s, events at 4.335s
Moved to previous frame: markers at 3.970s, events at 4.325s (delay: 0.355s)
Moved to previous frame: markers at 3.960s, events at 4.315s (delay: 0.355s)
Moved to previous frame: markers at 3.950s, events at 4.305s (delay: 0.355s)
Moved to previous frame: markers at 3.940s, events at 4.295s (delay: 0.355s)
Moved to previous frame: markers at 3.930s, events at 4.285s (delay: 0.355s)
Moved to previous frame: markers at 3.920s, events at 4.275s (delay: 0.355s)
Delay step increased to: 0.011s
Delay step increased to: 0.012s
Delay step increased to: 0.013s
Delay step increased to: 0.014s
Delay step decreased to: 0.013s
Delay step decreased to: 0.012s
Delay step decreased to: 0.011s
Delay step decreased to: 0.010s
Delay step decreased to: 0.009s
Delay step decreased to: 0.008s
Delay step decreased to: 0.007s
Delay s

In [ ]:
### Labeling with projection of the points on the event camera plane

In [ ]:
importlib.reload(helpers)
from helpers import DvsLabeler

# Create corrector instance
corrector = DvsLabeler(img_shape=(720, 1280, 3))

# Batch correction across multiple frames
time_tags, event_indices = helpers.calc_indices(e_ts, period)

# Define marker names for correction
marker_names = ['monitor:Origin_mon_top_right  ',
       'monitor:mon_bottom_right      ', 'monitor:mon_bottom_left       ',
       'monitor:mon_top_left          ', 'monitor:base_origin_1         ',
       'monitor:base_2                ', 'monitor:base_3                ',
       'monitor:base_4                ', 'chair:origin_right_top        ',
       'chair:right_mid               ', 'chair:right_bottom            ',
       'chair:left_top                ', 'chair:left_bottom             ',
       'box2:top_right_Origin         ', 'box2:bottom_right             ',
       'box2:bottom_left              ', 'box2:top_left                 ',
       'box1:top_1_origin             ', 'box1:top_2                    ',
       'box1:top_3                    ', 'box1:top_4                    ',
       'box1:bottom_1                 ', 'box1:bottom_2                 ',
       'box1:bottom_3                 ', 'box1:bottom_4                 ']

# Call correct_data for batch correction
corrected_data_success, process_continue, corrected_results, final_img = corrector.correct_data(
    e_ts, e_us, e_vs, event_indices, time_tags, period, 
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, D
)

if corrected_data_success and corrected_results:
    print(f"Batch correction completed. Corrected {len(corrected_results)} frames.")
    # Save corrected results if needed
    correction_path = os.path.join(os.path.dirname(vicon_c3d_file), 'corrections_labels_left.yml')
    # You can implement a save function similar to save_labeled_points if needed
else:
    print("Batch correction was cancelled or failed.")

cv2.destroyAllWindows()

space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, s

In [16]:
new_events_path = '/data/test6/left'
new_vicon_c3d_file: str = '/data/test6/VICON/test6.c3d'
 
new_v_data: dict = importIitYarp(filePathOrName=new_events_path, template=None, wrapTime=0.0)
new_e_ts: np.ndarray = new_v_data['data']['left']['dvs']['ts']
new_e_us: np.ndarray = new_v_data['data']['left']['dvs']['x']
new_e_vs: np.ndarray = new_v_data['data']['left']['dvs']['y']
 
new_c3d_data = c3d.Reader(open(vicon_c3d_file, 'rb'))
new_points_3d: dict = {}
for i, points, analog in new_c3d_data.read_frames():
    new_points_3d[i] = points
 
new_marker_t = np.linspace(0.0, new_c3d_data.frame_count / new_c3d_data.point_rate,
                            new_c3d_data.frame_count, endpoint=False)


importIitYarp trying path: /data/test6/left


100%|██████████| 93876/93876 [03:19<00:00, 469.39it/s]


Examining info.log: /data/test6/left/info.log


In [38]:
importlib.reload(helpers)
from helpers import ViconHelper

# Define vicon helper
new_vicon_helper = ViconHelper(new_marker_t, new_points_3d, delay, new_c3d_data.frame_count, new_c3d_data.point_rate, new_c3d_data.point_labels, True, True)

new_Ts_world_to_system = new_vicon_helper.compute_camera_marker_transforms()

out {'points': [{'stereoatis:cam_right': array([-436.5234 ,  790.85315,  879.18604], dtype=float32), 'stereoatis:cam_back': array([-494.32202,  855.09015,  919.47534], dtype=float32), 'stereoatis:cam_left': array([-479.50995,  807.1114 ,  857.0858 ], dtype=float32)}, {'stereoatis:cam_right': array([-436.71155,  790.6289 ,  878.95746], dtype=float32), 'stereoatis:cam_back': array([-494.32016,  855.094  ,  919.4863 ], dtype=float32), 'stereoatis:cam_left': array([-479.55362,  807.0915 ,  857.0675 ], dtype=float32)}, {'stereoatis:cam_right': array([-436.28622,  791.00055,  879.47644], dtype=float32), 'stereoatis:cam_back': array([-494.31198,  855.1035 ,  919.4934 ], dtype=float32), 'stereoatis:cam_left': array([-479.54333,  807.1374 ,  857.12134], dtype=float32)}, {'stereoatis:cam_right': array([-436.49933,  790.81195,  879.1287 ], dtype=float32), 'stereoatis:cam_back': array([-494.31723,  855.12244,  919.48663], dtype=float32), 'stereoatis:cam_left': array([-479.5291 ,  807.18335,  857.1

In [40]:
new_projected_points = helpers.project_vicon_to_event_plane_dynamic(
    marker_names, new_c3d_data, new_points_3d, new_marker_t, T_w2s_opt, new_Ts_world_to_system, K, cam_res, delay, new_e_ts, new_e_us, new_e_vs, period, visualize=True, D=D
)